# __Kaggle Project 2 - House Prices__

#### Nick Tedesco

## **Package and Data Loading**

In [45]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [61]:
train = pd.read_csv("train.csv")

train.head(n = 10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [64]:
test = pd.read_csv("test.csv")

First, let's combine the data for joint preprocessing. 

In [65]:
## extract outcome variable from training data
train.Y = train[['Id', 'SalePrice']]

## label training and testing datasets
train['train'] = "yes"
test['train'] = "no"

## drop outcome for train and combine with test
data = pd.concat([train.drop(columns = 'SalePrice'), test])

## **Data Preprocessing**

Let's start by taking a look at our distribution of missing values.

In [66]:
na_prop = data.isna().sum() / len(data)

na_prop.sort_values(ascending = False)[na_prop > 0]

PoolQC          0.996574
MiscFeature     0.964029
Alley           0.932169
Fence           0.804385
FireplaceQu     0.486468
LotFrontage     0.166495
GarageFinish    0.054471
GarageQual      0.054471
GarageCond      0.054471
GarageYrBlt     0.054471
GarageType      0.053786
BsmtExposure    0.028092
BsmtCond        0.028092
BsmtQual        0.027749
BsmtFinType2    0.027407
BsmtFinType1    0.027064
MasVnrType      0.008222
MasVnrArea      0.007879
MSZoning        0.001370
Functional      0.000685
Utilities       0.000685
BsmtHalfBath    0.000685
BsmtFullBath    0.000685
GarageArea      0.000343
BsmtFinSF1      0.000343
SaleType        0.000343
GarageCars      0.000343
BsmtUnfSF       0.000343
Electrical      0.000343
Exterior2nd     0.000343
Exterior1st     0.000343
KitchenQual     0.000343
TotalBsmtSF     0.000343
BsmtFinSF2      0.000343
dtype: float64

After reading the data description, many variables use NA to indicate that the house does not have a given feature. For example, NA is used to indicate "No pool" for the PoolQC variable. Therefore, we will simply replace NA values with "None" for each of the applicable variables. Similarly, continuous variables for features that are not applicable to the given house will be filled in with 0. 

It seems as if LotFrontage is the only continuous variable that requires a different imputation strategy (since NA doesn't indicate that the house is missing lot frontage!). Also, there are some categorical variables (Eletrical, Utilities) that should also be dealt with in a different manner.

Listed below are the four types (Type 1 - 4) of methods we will use to address missing values in this dataset. 

1. for LotFrontage, we will fill in missing values with the mean of the variable
2. for all categorical variables describing missing features, we will fill in missing values with a new category: "None"
3. for all continuous variables describing missing features, we will fill in missing values with zeroes
4. for the remaining categorical variables, where missing values do not indicate missing features (such as Electrical or Utilities), we will fill in values with the mode of the variable

In [81]:
## Type 1
continuous_missing_columns = ['LotFrontage']

for col in continuous_missing_columns:
    data[col].fillna(value = data[col].mean(), inplace = True)

## Type 2
na_equals_none_columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageYrBlt', 'GarageType', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'BsmtFinType1', 'MasVnrType']

for col in na_equals_none_columns:
    data[col].fillna(value = "None", inplace = True)

## Type 3
na_equals_zero_columns = ['MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'GarageArea', 'BsmtFinSF1', 'GarageCars', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFinSF2']

for col in na_equals_zero_columns:
    data[col].fillna(value = 0, inplace = True)

## Type 4
categorical_missing_columns = ['MSZoning', 'Functional', 'Utilities', 'SaleType', 'Electrical', 'Exterior1st', 'Exterior2nd', 'KitchenQual']

for col in categorical_missing_columns:
    data[col].fillna(value = data[col].mode(), inplace = True)

In [83]:
data[data['MSZoning'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,train
455,1916,30,NaN,109.000000,21780,Grvl,None,Reg,Lvl,NaN,...,0,None,None,None,0,3,2009,ConLD,Normal,no
756,2217,20,NaN,80.000000,14584,Pave,None,Reg,Low,AllPub,...,0,None,None,None,0,2,2008,WD,Abnorml,no
790,2251,70,NaN,69.305795,56600,Pave,None,IR1,Low,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,no
1444,2905,20,NaN,125.000000,31250,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2006,WD,Normal,no


Finally, let's check to make sure all of the missing values have been dealt with!

In [82]:
na_prop = data.isna().sum() / len(data)

na_prop.sort_values(ascending = False)[na_prop > 0]

MSZoning       0.001370
Utilities      0.000685
Functional     0.000685
Electrical     0.000343
KitchenQual    0.000343
Exterior1st    0.000343
Exterior2nd    0.000343
SaleType       0.000343
dtype: float64